Run the cell below if you are using Google Colab to mount your Google Drive in your Colab instance. Adjust the path to the files in your Google Drive as needed if it differs.

If you do not use Google Colab, running the cell will simply do nothing, so do not worry about it.

In [0]:
try:
    from google.colab import drive
    drive.mount('/content/drive/')
    %cd 'drive/My Drive/Colab Notebooks/01_Collaborative_Filtering'
except ImportError as e:
    pass

# **Exercise 1: Recommender Systems - Collaborative Filtering**

Collaborative filtering is a technique used in recommendation systems to predict a user's preference for a particular item based on the preferences of other similar users.

The idea behind collaborative filtering is that if two users have similar preferences for a set of items, they are likely to have similar preferences for other items as well. Similarly, if two items have been rated highly by the same set of users, they are likely to be similar in some way and may be recommended to users who have shown an interest in one of the items.

Collaborative filtering algorithms can be based on either user-based or item-based methods. User-based collaborative filtering recommends items to a user based on the preferences of other users who are similar to them, while item-based collaborative filtering recommends items that are similar to the items a user has previously liked or rated highly.

![image.png](attachment:image.png)
[Source: A collaborative filtering recommendation system with dynamic time decay](https://doi.org/10.1007/s11227-020-03266-2)

# **The surprise library**

[Surprise](https://surpriselib.com/) is a Python [scikit](https://projects.scipy.org/scikits.html) for building and analyzing recommender systems that deal with explicit rating data.

The name SurPRISE (roughly :) ) stands for Simple Python RecommendatIon System Engine.

In [0]:
# Install the surprise package
!conda install -c conda-forge scikit-surprise

/bin/bash: conda: command not found


**MovieLens**

The dataset [**MovieLens-100k**](https://grouplens.org/datasets/movielens/) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. 
It contains:
- 100,836 ratings
- 3,683 tag applications 
- 9,742 movies
- 610 users

These data were created between March 29, 1996 and September 24, 2018.

In [0]:
!pip install surprise
from surprise import Dataset

# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

# Retrieve the trainset.
trainset = data.build_full_trainset()

     |████████████████████████████████| 771 kB 6.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3177232 sha256=aaf8ba35629ddc153dbb955dd8f10f23f929b2ae9d89d58a8070b9aec78dc8d3
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b9e912b4-e1ee-4ccd-b117-23160487706b/bin/python -m pip install --upgrade pip' command.


/databricks/python/lib/python3.9/site-packages/pkg_resources/__init__.py:122: PkgResourcesDeprecationWarning: 0.23ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y

Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


![image-2.png](attachment:image-2.png)

We are interested in the rating of user 196 for the movie "Queen Margot" with the id 302

In [0]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item(movie) id (as in the ratings file). They are **strings**!

**[Similarity measure configuration](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measures-configuration)**

Many algorithms use a similarity measure to estimate a rating. The way they can be configured is done in a similar fashion as for baseline ratings: you just need to pass a `sim_options` argument at the creation of an algorithm. This argument is a dictionary with the following (all optional) keys:

 `name`: The name of the similarity to use, as defined in the similarities module. Options from the lecture: `pearson` and `cosine`. Default is `MSD`.

 `user_based`: Whether similarities will be computed between users or between items. This has a huge impact on the performance of a prediction algorithm. Default is True.

  `min_support`: The minimum number of common items (when `user_based` is `True`) or minimum number of common users (when `user_based` is `False`) for the similarity not to be zero.

In [0]:
# Define a custom similarity function for the KNNBasic algorithm
sim_options = {'name': 'pearson', 'user_based': True, 'min_support': 1}

**[KNNBasic](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic)** is a basic collaborative filtering algorithm for the simple prediction.

The prediction is set as:

![image.png](attachment:image.png)

or

![image-2.png](attachment:image-2.png)

depending on the `user_based` field of the `sim_options` parameter.

In [0]:
from surprise import KNNBasic
# User-based with Pearson correlation similarity and simple prediction
from surprise import KNNBasic

# User-based with Pearson correlation similarity and simple prediction

# Build algorithm
algo = KNNBasic(k=40, sim_options=sim_options)
algo.fit(trainset)

# Get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

# Build algorithm
#algo = KNNBasic(sim_options=sim_options, k)
#algo.fit(trainset)

# get a prediction for specific users and items.
#pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 3.24   {'actual_k': 30, 'was_impossible': False}


[KNNWithMeans](https://surprise.readthedocs.io/en/stable/knn_inspired.html#surprise.prediction_algorithms.knns.KNNBasic) is a basic collaborative filtering algorithm, taking into account the mean ratings of each user.

The prediction is set as:
![image-3.png](attachment:image-3.png)

or
![image-4.png](attachment:image-4.png)

depending on the `user_based` field of the `sim_options` parameter.

In [0]:
from surprise import KNNWithMeans
# User-based with Pearson correlation similarity and adjusted prediction considering the rating behaviour

# Define a custom similarity function for the KNNBasic algorithm
sim_options = {'name': 'pearson', 'user_based': True, 'min_support': 1}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 4.03   {'actual_k': 40, 'was_impossible': False}


In [0]:
# Item-based with Cosine similarity and simple prediction

# Define a custom similarity function for the KNNBasic algorithm
sim_options = {'name': 'cosine', 'user_based': False, 'min_support': 1}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 3.61   {'actual_k': 39, 'was_impossible': False}


In [0]:
# Item-based with Pearson (Adjusted Cosine) similarity and simple prediction

# Define a custom similarity function for the KNNBasic algorithm
sim_options = {'name': 'pearson', 'user_based': False, 'min_support': 1}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=4, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
user: 196        item: 302        r_ui = 4.00   est = 3.24   {'actual_k': 30, 'was_impossible': False}
